In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy
import datetime
import os

In [45]:
sampledpixels = pd.read_csv('/gpfs/glad3/HLSDIST/Validation/LandCoverBased/sampledpixels0110.csv')
numpy.set_printoptions(precision=5)
#print(sampledpixels[['ID'==1]])

annmin ={}
with open ('/gpfs/glad3/HLSDIST/Validation/LandCoverBased/analysis/mapLabels2023/All_3yrAnnMin_2023.csv','r') as dat:
  lines = dat.readlines()[1:]
  for line in lines:
    (ID,Stratum,Long,Lat,minvf) = line.strip().split(',')
    try:
      annmin[ID] = int(minvf)
    except:
      annmin[ID] = 0

nocountlimit = 2
nodaylimit = 15

table ={}
with open ('../sampledpixels0110.csv','r') as dat:
  lines = dat.readlines()[1:]
  allIDs = []
  for line in lines:
    (ID,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = line.strip().split(',')
    table[ID] = line.strip()
    allIDs.append(ID)

In [41]:
def getTS(i,name):
    index = str(i)

    df = pd.read_csv('/gpfs/glad3/HLSDIST/Validation/LandCoverBased/analysis/mapLabels2023/VF/'+index+'_VF_'+name+'.csv',names=['SensingTime','VBaselineCount','VBaseline','VEG-IND','VEG-ANOM'])

    #date_time = pd.to_datetime(df.SensingTime,format="%Y%m%dT%H%M%S")
    df['VBaseline'][df['VBaseline'].isnull()] = 255
    df[['VEG-IND','VEG-ANOM','VBaseline','VBaselineCount']]= df[['VEG-IND','VEG-ANOM','VBaseline','VBaselineCount']].astype(int)
    #df['VEG-IND'] = df['VBaseline']-df['VEG-ANOM']
    if annmin[index] > 85:
        df['VEG-ANOM'][df['VBaseline']==255] = df['VBaseline'][df['VBaseline']==255] -df['VEG-ANOM'][df['VBaseline']==255]
    #df['VEG-ANOM'][df['VBaseline'].isnull()] = 255
    

    return df[['SensingTime','VEG-IND','VEG-ANOM','VBaseline','VBaselineCount']]#.rename(columns={'SensingTimeMonthDay':'SensingTime'})


In [42]:
def updateLayers(currVF,currAnom,currDate,status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF):
    #statusLabels = {'nodata':255,'nodist':0,'lodloc':1,'lodhic':2,'hidloc':3,'hidhic':4,'lodlocdone':5,'lodhicdone':6,'hidlocdone':7,'hidhicdone':8}
    statusLabels = {'nodata':255,'nodist':0,'firstlo':1,'provlo':2,'conflo':3,'firsthi':4,'provhi':5,'confhi':6,'finlo':7,'finhi':8}
    lowthresh=10
    highthresh=50

    if(currAnom<=100):
        #lastObs = currDate
        if((currDate - date)>365 or (status>6 and currAnom>=lowthresh)):
            status=statusLabels['nodist']
            percent=200 
            count=0
            maxi=0
            conf=0
            date=0
            dur=0
            histVF=200

        prevcount = count
        if(percent>0 and percent<=100):
            prevnocount = int(((100-percent)/percent)*prevcount)
        else:
            prevnocount=0
            
        if(currAnom>=lowthresh):
            if(status==statusLabels['nodist'] or status==statusLabels['nodata']):# or status>4):
                date = currDate
                maxi = currAnom
                histVF = currVF + currAnom
                percent=100
                count=1
            else:#status is 1-6
                if(currAnom > maxi):
                    maxi = currAnom
                    histVF = currVF + currAnom
                if(count<253):
                    count+=1
                    percent = int((count*100)/(count + prevnocount))

            dur = currDate - date + 1
            #if(currAnom>5):
            #    nocount=0   

        else:#no current anomaly but valid obs
            if(percent>0 and percent<=100 and count<254):
                percent = int((count*100)/(prevcount + prevnocount + 1))
                #nocount+=1
            if(status==statusLabels['nodata']):#if was no data
                status=statusLabels['nodist']
                percent=200 
                count=0
                nocount=0
                maxi=0
                conf=0
                date=0
                dur=0
                histVF=200
        if(conf>0 and status<=6):
            prevmean = conf/(prevcount*prevcount)
            mean = (prevmean * (prevcount + prevnocount) + currAnom)/(prevcount + prevnocount + 1)
            tempconf = mean * count * count
            if(tempconf > 32000):
                conf=32000
            else:
                conf=int(tempconf)
        elif(status==0 or status==statusLabels['nodata']):
            if currAnom>=lowthresh:
                conf = currAnom
            else:
                conf=0

        lastAnomDate = date+dur-1
        nocount = int(lastObs>lastAnomDate) + int(currAnom<lowthresh)
        if(status<=6 or status==statusLabels['nodata']):
            #if(count==1 and nocount>=2):
            #    status=statusLabels['nodist']
            #    percent=200 
            #    count=0
            #    nocount=0
            #    maxi=0
            #    conf=0
            #    date=0
            #    dur=0
            #    histVF=200
            #el
            if((nocount>=nocountlimit or ((currDate-lastAnomDate)>=nodaylimit and lastAnomDate >0)) or (count==1 and currAnom<lowthresh)):# and currAnom/maxi < 0.5)):#reset if there was a single date anomaly#reset if no anom count >=5 (this doesn't have to be consecutive...)
                if status == 3:
                    status = statusLabels['finlo']
                elif status == 6:
                    status = statusLabels['finhi']
                else:
                    status=statusLabels['nodist']
                    percent=200 
                    count=0
                    nocount=0
                    maxi=0
                    conf=0
                    date=0
                    dur=0
                    histVF=200
            elif(maxi>=highthresh):
                if(conf>=400):
                    status=6 #if(percent>=75 and count>=3):status=4}
                elif dur==1:
                    status=statusLabels['firsthi']
                elif(status!=6):
                    status=5
            elif(maxi>=lowthresh):
                if(conf>=400):
                    status=3#if(percent>=75 and count>=4):status=2}
                elif dur==1:
                    status=statusLabels['firstlo']
                elif(status!=3):
                    status=2
            else:
                status=statusLabels['nodist']
        lastObs = currDate
    return (status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF)

In [5]:
def createLayers(i,name):
    source = getTS(i,name)
    (status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF) = (255,255,-1,-1,255,255,255,-1,-1,255) 
    #print('currVF','currAnom','currDate','status','maxi','conf','date','count','nocount','percent','dur','lastObs','histVF')
    #df = pd.DataFrame(columns=['SensingTime','currVF','currAnom','currDate','status','maxi','conf','date','count','nocount','percent','dur','lastObs','histVF'])
    df = pd.DataFrame(columns=['SensingTime','VEG-IND','VEG-ANOM','currDate','status','maxi','VEG-DIST-CONF','date','count','nocount','percent','dur','lastObs','histVF','VBaseline','VBaselineCount'])
    for index, row in source.iterrows():
        #print(row)
        startdate = datetime.datetime.strptime("2023001","%Y%j")
        enddate = datetime.datetime.strptime(row['SensingTime'],"%Y%jT%H%M%S")
        days = enddate-startdate
        currDate = days.days+1
        currVF = row['VEG-IND']
        currAnom = row['VEG-ANOM']
        #if annmin[i] > 85 and currAnom == 255:
        #    print(i,annmin[ID],currVF,currAnom,currDate)
        (status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF) =updateLayers(currVF,currAnom,currDate,status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF)
        df.loc[len(df.index)] = [row['SensingTime'],currVF,currAnom,currDate,status,maxi,conf,date,count,nocount,percent,dur,lastObs,histVF,row['VBaseline'],row['VBaselineCount']]
    return df
    #print(df)
createLayers("1386","3_15")

/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,SensingTime,VEG-IND,VEG-ANOM,currDate,status,maxi,VEG-DIST-CONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount
0,2023001T084149,9,-12,1,0,0,0,0,0,1,200,0,1,200,13,15
1,2023001T084249,9,-11,1,0,0,0,0,0,0,200,0,1,200,13,15
2,2023006T084341,9,-11,6,0,0,0,0,0,0,200,0,6,200,13,16
3,2023009T084150,9,-8,9,0,0,0,0,0,0,200,0,9,200,13,16
4,2023011T084229,9,-12,11,0,0,0,0,0,0,200,0,11,200,10,17
5,2023016T084311,9,-12,16,0,0,0,0,0,0,200,0,16,200,10,21
6,2023017T084140,9,-11,17,0,0,0,0,0,0,200,0,17,200,10,21
7,2023021T084159,9,-11,21,0,0,0,0,0,0,200,0,21,200,10,22
8,2023025T084146,9,-11,25,0,0,0,0,0,0,200,0,25,200,10,20
9,2023031T084109,9,-11,31,0,0,0,0,0,0,200,0,31,200,10,22


In [18]:
nocountlimit = 2
nodaylimit = 15
for name in ["3_10","3_25","1_15","2_15","4_15","5_15"]:
    filename="mapLabels2023/TStables_"+name
    os.makedirs(filename, exist_ok =True)
    #os.mkdir(name+"_plots")
    ann = pd.DataFrame(columns=["ID","STATUS","ANOM-MAX","CONF","DATE","COUNT","DUR","VEG-HIST","IND-MAX"])
    for i in range(1,1651):
        try:
            df = createLayers(i,name)
            df.to_csv(filename+"/"+str(i)+"_"+name+".csv")
        except:
            print(i)
        #makeChart(name,i,df)

/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

546


/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


546


/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

...

/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

546


/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

546


/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/apps/python/3.7/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [47]:
#allIDs = list(range(1,546))+list(range(547,1651))
strataarea = {}
with open("Share/area-DIST-accuracy/stratatable_nosplit.txt",'r') as stratatable:
  lines = stratatable.readlines()[1:]
  for l in lines:
    (stratum,count,area,K,zcount) = l.split('\t')
    strataarea[int(stratum)] = float(area)
    Nstrata = int(stratum)

changetype = {}
conversion = {}
with open("Share/area-DIST-accuracy/reference_conversion.csv",'r') as tablecsv:
  lines = tablecsv.readlines()[1:]
  for l in lines:
    (ID,changelabel,conversionlabel,proportion,prevyear,overallLabel,Long,Lat,UTMzone,centxUTM,centyUTM) = l.split(',')[0:11]
    changetype[ID] = changelabel
    conversion[ID] = conversionlabel

In [19]:
def compareBaseline(Nyearlist,Ndaylist):
  data = {}
  alert = {}
  conf = {}
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      name = str(Nyears)+"_"+str(Ndays)
      #print(name)
      data[name] = {}
      alert[name] = {}
      conf[name] = {}
      for ID in allIDs:
        with open("mapLabels2023/TStables_"+name+"/"+str(ID)+"_"+str(Nyears)+"_"+str(Ndays)+".csv",'r') as csv:
          lines = csv.readlines()[1:]
          datasum=0
          alertsum=0
          confsum=0
          for l in lines:
            (index,SensingTime,VEGIND,VEGANOM,currDate,status,maxi,VEGDISTCONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount) = l.split(',')
            status = int(status)
            if status !=255:
              datasum+=1
              if status>0:
                alertsum+=1
              if status==3 or status>=6:
                confsum+=1
          data[name][ID] = datasum
          if datasum>0:
            alert[name][ID] = [alertsum,alertsum/datasum]
            conf[name][ID] = [confsum,confsum/datasum]
          else:
            alert[name][ID] = [alertsum,0]
            conf[name][ID] = [confsum,0]
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      l = [0 for i in range(0,Nstrata+1)]
      g = [0 for i in range(0,Nstrata+1)]
      b = [0 for i in range(0,Nstrata+1)]
      t = [0 for i in range(0,Nstrata+1)]
      area = 0
      lossarea = 0
      gainarea = 0
      name = str(Nyears)+"_"+str(Ndays)
      with open("mapLabels2023/_"+name+"delta_3_15.csv","w") as outfile:
      #  outfile.write("ID,datacount,alertpercent,confpercent\n")
        for ID in allIDs:
          #outfile.write(','.join([str(ID),str(data[name][ID]-data["3_15"][ID]),f'{(alert[name][ID][1]-alert["3_15"][ID][1]):.8f}',f'{(conf[name][ID][1]-conf["3_15"][ID][1]):.8f}'])+"\n")
      #    outfile.write(','.join([str(ID),str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0])])+"\n")
      #with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
          (ID,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = table[str(ID)].split(',')
          loss = (conf[name][ID][0]==0 and conf["3_15"][ID][0]>0)
          gain = (conf[name][ID][0]>0 and conf["3_15"][ID][0]==0)
          l[int(Stratum)]+=loss
          g[int(Stratum)]+=gain
          b[int(Stratum)]+=(conf["3_15"][ID][0]>0)
          t[int(Stratum)]+=(data[name][ID]>0)
          
          outfile.write(','.join([str(ID),Stratum,str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0]),changetype[ID],conversion[ID]])+"\n")
      with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
        for i in range(0,Nstrata+1):
          if t[i] > 0:
            tarea = (b[i]+g[i]-l[i])/t[i]*strataarea[i]
            area += tarea
            larea = l[i]/t[i]*strataarea[i]
            garea = g[i]/t[i]*strataarea[i]
            lossarea -=larea
            gainarea +=garea
          else:
            tarea = 0
            larea = 0
            garea = 0
          outfile.write(str(i)+','+str(l[i])+','+str(g[i])+','+str(b[i])+","+f'{larea:.2f}'+","+f'{garea:.2f}'+","+f'{tarea:.2f}'+"\n")
      print(name,area,lossarea,gainarea,f'{lossarea*100/4937817.513779193:.2f}',f'{gainarea*100/4937817.513779193:.2f}')

compareBaseline([3],[5,10,15,20,25,30])
compareBaseline([1,2,3,4,5],[15])



3_5 5886143.230030409 -1292301.9485832 2173111.0631927536 -26.17 44.01
3_10 6108718.309310676 -362772.4841384616 1530346.7291760654 -7.35 30.99
3_15 4937817.513779193 0.0 0.0 0.00 0.00
3_20 4324525.760076866 -794291.8176905705 181000.06398824116 -16.09 3.67
3_25 3639203.780535706 -1650821.5927846704 352207.85954118235 -33.43 7.13
3_30 3109106.399406194 -2182679.7739925827 353968.6596195824 -44.20 7.17
1_15 11613512.002036402 -805665.4399736789 7454264.87720875 -16.32 150.96
2_15 9154837.416179828 -248813.17960211023 4460859.2883861605 -5.04 90.34
3_15 4937817.513779193 0.0 0.0 0.00 0.00
4_15 3176628.890940206 -1854392.7499328002 93204.12709381175 -37.55 1.89
5_15 2385043.154882835 -2626973.351945771 74198.99304941176 -53.20 1.50


In [54]:
def compareBaselineData(Nyearlist,Ndaylist):
  data = {}
  alert = {}
  conf = {}
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      name = str(Nyears)+"_"+str(Ndays)
      #print(name)
      data[name] = {}
      alert[name] = {}
      conf[name] = {}
      for ID in allIDs:
        with open("mapLabels2023/TStables_"+name+"/"+str(ID)+"_"+str(Nyears)+"_"+str(Ndays)+".csv",'r') as csv:
          lines = csv.readlines()[1:]
          datasum=0
          alertsum=0
          confsum=0
          for l in lines:
            (index,SensingTime,VEGIND,VEGANOM,currDate,status,maxi,VEGDISTCONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount) = l.split(',')
            status = int(status)
            if int(VEGANOM) !=255:
              datasum+=1
              if status>0 and status !=255:
                alertsum+=1
              if (status==3 or status>=6) and status !=255:
                confsum+=1
          data[name][ID] = datasum
          if datasum>0:
            alert[name][ID] = [alertsum,alertsum/datasum]
            conf[name][ID] = [confsum,confsum/datasum]
          else:
            alert[name][ID] = [alertsum,0]
            conf[name][ID] = [confsum,0]
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      l = [0 for i in range(0,Nstrata+1)]
      g = [0 for i in range(0,Nstrata+1)]
      b = [0 for i in range(0,Nstrata+1)]
      t = [0 for i in range(0,Nstrata+1)]
      area = 0
      lossarea = 0
      gainarea = 0
      name = str(Nyears)+"_"+str(Ndays)
      with open("mapLabels2023/_"+name+"delta_3_15.csv","w") as outfile:
      #  outfile.write("ID,datacount,alertpercent,confpercent\n")
        for ID in allIDs:
          #outfile.write(','.join([str(ID),str(data[name][ID]-data["3_15"][ID]),f'{(alert[name][ID][1]-alert["3_15"][ID][1]):.8f}',f'{(conf[name][ID][1]-conf["3_15"][ID][1]):.8f}'])+"\n")
      #    outfile.write(','.join([str(ID),str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0])])+"\n")
      #with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
          (ID2,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = table[str(ID)].split(',')
          loss = (data[name][ID]==0 and data["3_15"][ID]>0)
          gain = (data[name][ID]>0 and data["3_15"][ID]==0)
          l[int(Stratum)]+=loss
          g[int(Stratum)]+=gain
          b[int(Stratum)]+=(data["3_15"][ID]>0)
          t[int(Stratum)]+=(data[name][ID]>0)
          
          outfile.write(','.join([str(ID),Stratum,str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0]),changetype[ID],conversion[ID]])+"\n")
      with open("mapLabels2023/_"+name+"deltadata_3_15.csv","w") as outfile:
        for i in range(0,Nstrata+1):
          if t[i] > 0:
            tarea = (b[i]+g[i]+l[i])/t[i]*strataarea[i]
            area += tarea
            larea = l[i]/t[i]*strataarea[i]
            garea = g[i]/t[i]*strataarea[i]
            lossarea -=larea
            gainarea +=garea
          else:
            tarea = 0
            larea = 0
            garea = 0
          outfile.write(str(i)+','+str(l[i])+','+str(g[i])+','+str(b[i])+","+f'{larea:.2f}'+","+f'{garea:.2f}'+","+f'{tarea:.2f}'+"\n")
      print(name,area,lossarea,gainarea,f'{lossarea*100/132069533.781995:.2f}',f'{gainarea*100/132069533.781995:.2f}')

compareBaselineData([3],[5,10,15,20,25,30])
compareBaselineData([1,2,3,4,5],[15])



3_5 137138645.28709516 -2534555.752550072 0.0 -1.92 0.00
3_10 132082839.98397051 -6653.100987755101 0.0 -0.01 0.00
3_15 132069533.781995 0.0 0.0 0.00 0.00
3_20 132069533.781995 0.0 0.0 0.00 0.00
3_25 132069533.781995 0.0 0.0 0.00 0.00
3_30 132069533.781995 0.0 0.0 0.00 0.00
1_15 132727075.34138907 -328770.77969703055 0.0 -0.25 0.00
2_15 132176615.65650773 -53540.93725636734 0.0 -0.04 0.00
3_15 132069533.781995 0.0 0.0 0.00 0.00
4_15 132069533.781995 0.0 0.0 0.00 0.00
5_15 132069533.781995 0.0 0.0 0.00 0.00


In [63]:
def compareBaselineCount(Nyearlist,Ndaylist):
  data = {}
  alert = {}
  conf = {}
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      name = str(Nyears)+"_"+str(Ndays)
      #print(name)
      data[name] = {}
      alert[name] = {}
      conf[name] = {}
      for ID in allIDs:
        with open("mapLabels2023/TStables_"+name+"/"+str(ID)+"_"+str(Nyears)+"_"+str(Ndays)+".csv",'r') as csv:
          lines = csv.readlines()[1:]
          datasum=0
          alertsum=0
          confsum=0
          for l in lines:
            (index,SensingTime,VEGIND,VEGANOM,currDate,status,maxi,VEGDISTCONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount) = l.split(',')
            status = int(status)
            if int(VEGANOM) !=255:
              datasum+=1
              if status>0 and status !=255:
                alertsum+=1
              if (status==3 or status>=6) and status !=255:
                confsum+=1
          data[name][ID] = datasum
          if datasum>0:
            alert[name][ID] = [alertsum,alertsum/datasum]
            conf[name][ID] = [confsum,confsum/datasum]
          else:
            alert[name][ID] = [alertsum,0]
            conf[name][ID] = [confsum,0]
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      l = [0 for i in range(0,Nstrata+1)]
      g = [0 for i in range(0,Nstrata+1)]
      m = [0 for i in range(0,Nstrata+1)]
      ng = [0 for i in range(0,Nstrata+1)]
      b = [0 for i in range(0,Nstrata+1)]
      t = [0 for i in range(0,Nstrata+1)]
      d = [0 for i in range(0,Nstrata+1)]
      area = 0
      lossarea = 0
      gainarea = 0
      globalarea=0
      tarea =0
      missingarea = 0
      newarea=0
      maxlost =0
      maxgained = 0
      avmaxloss = 0
      avmaxgain = 0
      stratagained = 0
      stratalost = 0
      name = str(Nyears)+"_"+str(Ndays)
      with open("mapLabels2023/_"+name+"delta_3_15.csv","w") as outfile:
      #  outfile.write("ID,datacount,alertpercent,confpercent\n")
        for ID in allIDs:
          #outfile.write(','.join([str(ID),str(data[name][ID]-data["3_15"][ID]),f'{(alert[name][ID][1]-alert["3_15"][ID][1]):.8f}',f'{(conf[name][ID][1]-conf["3_15"][ID][1]):.8f}'])+"\n")
      #    outfile.write(','.join([str(ID),str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0])])+"\n")
      #with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
          (ID2,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = table[str(ID)].split(',')
          loss = (data[name][ID]-data["3_15"][ID])*(data[name][ID]<data["3_15"][ID] and data["3_15"][ID]>0)
          missing = (data[name][ID]==0 and data["3_15"][ID]>0)
          gain = (data[name][ID]-data["3_15"][ID])*(data[name][ID]>data["3_15"][ID] and data["3_15"][ID]>0)
          newgain = (data[name][ID]>0 and data["3_15"][ID]==0)
          #gain = (data[name][ID]>0 and data["3_15"][ID]==0)
          l[int(Stratum)]+=loss
          m[int(Stratum)]+=missing
          g[int(Stratum)]+=gain
          ng[int(Stratum)]+=newgain
          b[int(Stratum)]+=1
          t[int(Stratum)]+=(data[name][ID]>0)
          d[int(Stratum)]+=(data[name][ID])
          
          outfile.write(','.join([str(ID),Stratum,str(data[name][ID]),str(data["3_15"][ID])])+"\n")
      with open("mapLabels2023/_"+name+"deltacount_3_15.csv","w") as outfile:
        for i in range(1,Nstrata+1):
          if t[i] > 0:
            tarea += d[i]/b[i]*strataarea[i]
            lossarea += l[i]/b[i]*strataarea[i]
            gainarea += g[i]/b[i]*strataarea[i]
            missingarea += m[i]/b[i]*strataarea[i]
            newarea += ng[i]/b[i]*strataarea[i]
            avcount = d[i]/b[i]
            area += tarea
            globalarea +=strataarea[i]
            #if l[i]/t[i] <maxlost:
            #  maxlost=l[i]/t[i]
            #  avmaxloss = maxlost/avcount
            #  stratalost = i
            #if g[i]/t[i] >maxgained:
            #  maxgained=g[i]/t[i]
            #  avmaxgain = maxgained/avcount
            #  stratagained = i
            if (l[i]/t[i])/avcount <avmaxloss:
              maxlost=l[i]/t[i]
              avmaxloss = maxlost/avcount
              stratalost = i
            if (g[i]/t[i])/avcount >avmaxgain:
              maxgained=g[i]/t[i]
              avmaxgain = maxgained/avcount
              stratagained = i
          else:
            #tarea = 0
            globalarea +=strataarea[i]
          #print(i,avcount,l[i]/t[i],g[i]/t[i])
          #outfile.write(str(i)+','+str(l[i])+','+str(g[i])+','+str(b[i])+","+f'{larea:.2f}'+","+f'{garea:.2f}'+","+f'{tarea:.2f}'+"\n")
      print(name,f'{tarea/globalarea:.2f}',f'{lossarea/globalarea:.2f}',f'{gainarea/globalarea:.2f}',f'{missingarea/globalarea*100:.2f}',f'{newarea/globalarea*100:.2f}',maxlost,maxgained,stratalost,stratagained,avmaxloss,avmaxgain)

compareBaselineCount([3],[5,10,15,20,25,30])
compareBaselineCount([1,2,3,4,5],[15])

3_5 45.83 -10.18 0.00 1.87 0.00 -17.46 0 33 0 -0.6679418515684774 0
3_10 54.05 -1.96 0.00 0.00 0.00 -3.96 0 33 0 -0.09989909182643794 0
3_15 56.01 0.00 0.00 0.00 0.00 0 0 0 0 0 0
3_20 56.88 0.00 0.88 0.00 0.00 0 1.42 0 33 0 0.03154153709462461
3_25 57.21 0.00 1.20 0.00 0.00 0 2.08 0 33 0 0.04553415061295972
3_30 57.36 0.00 1.35 0.00 0.00 0 2.22 0 33 0 0.04845045831514623
1_15 48.99 -7.01 0.00 0.24 0.00 -12.979591836734693 0 10 0 -0.41601255886970173 0
2_15 54.59 -1.42 0.00 0.04 0.00 -3.3 0 33 0 -0.08188585607940446 0
3_15 56.01 0.00 0.00 0.00 0.00 0 0 0 0 0 0
4_15 56.75 0.00 0.74 0.00 0.00 0 1.48 0 33 0 0.032830523513753325
5_15 57.09 0.00 1.08 0.00 0.00 0 2.0 0 33 0 0.043859649122807015


In [52]:
def compareBaselineDataGEN(Nyearlist,Ndaylist):
  data = {}
  alert = {}
  conf = {}
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      name = str(Nyears)+"_"+str(Ndays)
      #print(name)
      data[name] = {}
      alert[name] = {}
      conf[name] = {}
      for ID in allIDs:
        with open("mapLabels2023/TStables_"+name+"/"+str(ID)+"_"+str(Nyears)+"_"+str(Ndays)+".csv",'r') as csv:
          lines = csv.readlines()[1:]
          datasum=0
          alertsum=0
          confsum=0
          for l in lines:
            (index,SensingTime,VEGIND,VEGANOM,currDate,status,maxi,VEGDISTCONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount) = l.split(',')
            status = int(status)
            if int(VBaselineCount) >= 7:
              datasum+=1
          data[name][ID] = datasum
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      l = [0 for i in range(0,Nstrata+1)]
      g = [0 for i in range(0,Nstrata+1)]
      b = [0 for i in range(0,Nstrata+1)]
      t = [0 for i in range(0,Nstrata+1)]
      area = 0
      lossarea = 0
      gainarea = 0
      name = str(Nyears)+"_"+str(Ndays)
      with open("mapLabels2023/_"+name+"deltaGEN_3_15.csv","w") as outfile:
      #  outfile.write("ID,datacount,alertpercent,confpercent\n")
        for ID in allIDs:
          #outfile.write(','.join([str(ID),str(data[name][ID]-data["3_15"][ID]),f'{(alert[name][ID][1]-alert["3_15"][ID][1]):.8f}',f'{(conf[name][ID][1]-conf["3_15"][ID][1]):.8f}'])+"\n")
      #    outfile.write(','.join([str(ID),str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0])])+"\n")
      #with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
          (ID2,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = table[str(ID)].split(',')
          loss = (data[name][ID]==0 and data["3_15"][ID]>0)
          gain = (data[name][ID]>0 and data["3_15"][ID]==0)
          l[int(Stratum)]+=loss
          g[int(Stratum)]+=gain
          b[int(Stratum)]+=(data["3_15"][ID]>0)
          t[int(Stratum)]+=(data[name][ID]>0)
          
          outfile.write(','.join([str(ID),Stratum,str(data[name][ID]),str(data["3_15"][ID]),changetype[ID],conversion[ID]])+"\n")
      with open("mapLabels2023/_"+name+"deltadataGEN_3_15.csv","w") as outfile:
        for i in range(0,Nstrata+1):
          if t[i] > 0:
            tarea = (b[i]+g[i]-l[i])/t[i]*strataarea[i]
            area += tarea
            larea = l[i]/t[i]*strataarea[i]
            garea = g[i]/t[i]*strataarea[i]
            lossarea -=larea
            gainarea +=garea
          else:
            tarea = 0
            larea = 0
            garea = 0
          outfile.write(str(i)+','+str(l[i])+','+str(g[i])+','+str(b[i])+","+f'{larea:.2f}'+","+f'{garea:.2f}'+","+f'{tarea:.2f}'+"\n")
      print(name,area,lossarea,gainarea,f'{lossarea*100/132069533.781995:.2f}',f'{gainarea*100/132069533.781995:.2f}')

compareBaselineDataGEN([3],[5,10,15,20,25,30])
compareBaselineDataGEN([1,2,3,4,5],[15])

3_5 132069533.781995 -28182465.31437582 0.0 -21.34 0.00
3_10 132069533.781995 -3156655.4899004893 0.0 -2.39 0.00
3_15 132069533.781995 0.0 0.0 0.00 0.00
3_20 132069533.781995 0.0 536839.6474452801 0.00 0.41
3_25 132069533.781995 0.0 624159.0632016883 0.00 0.47
3_30 132069533.781995 0.0 628949.1304010352 0.00 0.48
1_15 132069533.781995 -16926898.426009607 0.0 -12.82 0.00
2_15 132069533.781995 -5129517.552254906 0.0 -3.88 0.00
3_15 132069533.781995 0.0 0.0 0.00 0.00
4_15 132069533.781995 0.0 337127.8362429944 0.00 0.26
5_15 132069533.781995 0.0 578620.1034527903 0.00 0.44


In [65]:
def compareBaselineCountGEN(Nyearlist,Ndaylist):
  data = {}
  alert = {}
  conf = {}
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      name = str(Nyears)+"_"+str(Ndays)
      #print(name)
      data[name] = {}
      alert[name] = {}
      conf[name] = {}
      for ID in allIDs:
        with open("mapLabels2023/TStables_"+name+"/"+str(ID)+"_"+str(Nyears)+"_"+str(Ndays)+".csv",'r') as csv:
          lines = csv.readlines()[1:]
          datasum=0
          alertsum=0
          confsum=0
          for l in lines:
            (index,SensingTime,VEGIND,VEGANOM,currDate,status,maxi,VEGDISTCONF,date,count,nocount,percent,dur,lastObs,histVF,VBaseline,VBaselineCount) = l.split(',')
            status = int(status)
            if int(VBaselineCount) >= 7:
              datasum+=1
            data[name][ID] = datasum
  for Ndays in Ndaylist:
    for Nyears in Nyearlist:
      l = [0 for i in range(0,Nstrata+1)]
      m = [0 for i in range(0,Nstrata+1)]
      g = [0 for i in range(0,Nstrata+1)]
      ng = [0 for i in range(0,Nstrata+1)]
      b = [0 for i in range(0,Nstrata+1)]
      t = [0 for i in range(0,Nstrata+1)]
      d = [0 for i in range(0,Nstrata+1)]
      area = 0
      lossarea = 0
      gainarea = 0
      globalarea=0
      tarea =0
      missingarea=0
      newarea=0
      maxlost =0
      maxgained = 0
      avmaxloss = 0
      avmaxgain = 0
      stratagained = 0
      stratalost = 0
      name = str(Nyears)+"_"+str(Ndays)
      with open("mapLabels2023/_"+name+"deltaGEN_3_15.csv","w") as outfile:
      #  outfile.write("ID,datacount,alertpercent,confpercent\n")
        for ID in allIDs:
          #outfile.write(','.join([str(ID),str(data[name][ID]-data["3_15"][ID]),f'{(alert[name][ID][1]-alert["3_15"][ID][1]):.8f}',f'{(conf[name][ID][1]-conf["3_15"][ID][1]):.8f}'])+"\n")
      #    outfile.write(','.join([str(ID),str(data[name][ID]),str(data["3_15"][ID]),str(alert[name][ID][0]),str(alert["3_15"][ID][0]),str(conf[name][ID][0]),str(conf["3_15"][ID][0])])+"\n")
      #with open("mapLabels2023/_"+name+"deltaconf_3_15.csv","w") as outfile:
          (ID2,Stratum,landcover,alertstrata,Long,Lat,zone,x,y,pixel,pline,centxUTM,centyUTM,MGRS,forestLossYear) = table[str(ID)].split(',')
          loss = (data[name][ID]-data["3_15"][ID])*(data[name][ID]<data["3_15"][ID] and data["3_15"][ID]>0)
          missing = (data[name][ID]==0 and data["3_15"][ID]>0)
          gain = (data[name][ID]-data["3_15"][ID])*(data[name][ID]>data["3_15"][ID] and data["3_15"][ID]>0)
          newgain = (data[name][ID]>0 and data["3_15"][ID]==0)
          #gain = (data[name][ID]>0 and data["3_15"][ID]==0)
          l[int(Stratum)]+=loss
          m[int(Stratum)]+=missing
          g[int(Stratum)]+=gain
          ng[int(Stratum)]+=newgain
          b[int(Stratum)]+=1
          t[int(Stratum)]+=(data[name][ID]>0)
          d[int(Stratum)]+=(data[name][ID])
          
          outfile.write(','.join([str(ID),Stratum,str(data[name][ID]),str(data["3_15"][ID])])+"\n")
      with open("mapLabels2023/_"+name+"deltacount_3_15.csv","w") as outfile:
        for i in range(1,Nstrata+1):
          if t[i] > 0:
            tarea += d[i]/b[i]*strataarea[i]
            lossarea += l[i]/b[i]*strataarea[i]
            gainarea += g[i]/b[i]*strataarea[i]
            missingarea += m[i]/b[i]*strataarea[i]
            newarea += ng[i]/b[i]*strataarea[i]
            avcount = d[i]/b[i]
            area += tarea
            globalarea +=strataarea[i]
            #if l[i]/t[i] <maxlost:
            #  maxlost=l[i]/t[i]
            #  avmaxloss = maxlost/avcount
            #  stratalost = i
            #if g[i]/t[i] >maxgained:
            #  maxgained=g[i]/t[i]
            #  avmaxgain = maxgained/avcount
            #  stratagained = i
            if (l[i]/t[i])/avcount <avmaxloss:
              maxlost=l[i]/t[i]
              avmaxloss = maxlost/avcount
              stratalost = i
            if (g[i]/t[i])/avcount >avmaxgain:
              maxgained=g[i]/t[i]
              avmaxgain = maxgained/avcount
              stratagained = i
          else:
            #tarea = 0
            globalarea +=strataarea[i]
          #print(i,avcount,l[i]/t[i],g[i]/t[i])
          #outfile.write(str(i)+','+str(l[i])+','+str(g[i])+','+str(b[i])+","+f'{larea:.2f}'+","+f'{garea:.2f}'+","+f'{tarea:.2f}'+"\n")
      print(name,f'{tarea/globalarea:.2f}',f'{lossarea/globalarea:.2f}',f'{gainarea/globalarea:.2f}',f'{missingarea/globalarea*100:.2f}',f'{newarea/globalarea*100:.2f}',maxlost,maxgained,stratalost,stratagained,avmaxloss,avmaxgain)

compareBaselineCountGEN([3],[5,10,15,20,25,30])
compareBaselineCountGEN([1,2,3,4,5],[15])

3_5 27.93 -24.99 0.00 16.89 0.00 -40.8421052631579 0 6 0 -4.862155388471178 0
3_10 48.32 -4.59 0.00 2.24 0.00 -10.63265306122449 0 6 0 -0.36639052588644005 0
3_15 52.92 0.00 0.00 0.00 0.00 0 0 0 0 0 0
3_20 54.64 0.00 1.72 0.00 0.40 0 4.24 0 33 0 0.10211946050096339
3_25 55.89 0.00 2.94 0.00 0.47 0 6.22 0 33 0 0.14298850574712643
3_30 56.49 0.00 3.53 0.00 0.48 0 7.82 0 33 0 0.17339246119733925
1_15 35.84 -17.07 0.00 10.50 0.00 -29.64516129032258 0 11 0 -2.5165671723533602 0
2_15 49.32 -3.60 0.00 3.62 0.00 -6.0 0 11 0 -0.24232633279483035 0
3_15 52.92 0.00 0.00 0.00 0.00 0 0 0 0 0 0
4_15 54.47 0.00 1.55 0.00 0.25 0 4.12 0 33 0 0.09951690821256039
5_15 55.48 0.00 2.55 0.00 0.44 0 6.2 0 33 0 0.1425942962281509
